# MedGemma-4b Clinical Report Generation

Generate clinical pathology reports from spatial transcriptomics data using MedGemma-4b-it (4-bit quantized).

**Input**: Spatial features + cell type annotations from Visium data  
**Output**: 200-word clinical pathology report  
**Model**: MedGemma-4b-it (4-bit quantization for M1 Mac 64GB)  

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"MPS available: {torch.backends.mps.is_available()}")
print(f"Device: {'mps' if torch.backends.mps.is_available() else 'cpu'}")

## 1. Load Spatial Analysis Results

In [ ]:
def load_analysis_data():
    """Load spatial features and cell type annotations."""
    with open('../outputs/spatial_features.json', 'r') as f:
        spatial_data = json.load(f)
    
    with open('../outputs/cell_type_enhanced_summary.json', 'r') as f:
        celltype_data = json.load(f)
    
    return spatial_data, celltype_data

spatial_features, celltype_summary = load_analysis_data()

print("Spatial Features Loaded:")
print(f"  Total spots: {spatial_features['dataset_info']['total_spots']}")
print(f"  Spatial clusters: {spatial_features['dataset_info']['n_clusters']}")
print(f"  Spatially variable genes: {spatial_features['spatial_statistics']['morans_i']['n_significant_genes']}")

print("\nCell Type Summary:")
total = celltype_summary['cell_type_stats']['total_spots']
for celltype, count in celltype_summary['cell_type_composition'].items():
    pct = (count / total) * 100
    print(f"  {celltype}: {count} ({pct:.1f}%)")

print(f"\nTumor-Immune Interface: {celltype_summary['tumor_immune_interface']['interface_pct']:.1f}%")

## 2. Configure and Load MedGemma-4b Model

In [ ]:
def load_medgemma_model():
    """Load MedGemma-4b-it with 4-bit quantization."""
    model_id = "google/medgemma-4b-it"
    
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )
    
    print(f"Loading {model_id} with 4-bit quantization...")
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True
    )
    
    print("Model loaded successfully")
    return tokenizer, model

tokenizer, model = load_medgemma_model()

## 3. Design Clinical Prompt Template

In [ ]:
def create_clinical_prompt(spatial_data, celltype_data):
    """Generate clinical pathology prompt from spatial analysis."""
    
    total_spots = celltype_data['cell_type_stats']['total_spots']
    composition = celltype_data['cell_type_composition']
    
    luminal_scgb = composition.get('LummHR-SCGB', 0)
    luminal_major = composition.get('LummHR-major', 0)
    plasma = composition.get('plasma_IgG', 0)
    
    luminal_pct = ((luminal_scgb + luminal_major) / total_spots) * 100
    plasma_pct = (plasma / total_spots) * 100
    
    interface_pct = celltype_data['tumor_immune_interface']['interface_pct']
    
    top_morans = list(spatial_data['spatial_statistics']['morans_i']['top_genes'].keys())[:5]
    
    top_luminal_markers = celltype_data['top_markers_per_celltype'].get('LummHR-SCGB', [])[:5]
    top_plasma_markers = celltype_data['top_markers_per_celltype'].get('plasma_IgG', [])[:5]
    
    prompt = f"""You are a board-certified pathologist reviewing spatial transcriptomics data from a breast cancer biopsy specimen. Based on the following molecular and spatial findings, generate a concise clinical pathology report (approximately 200 words).

SPECIMEN DATA:
- Total analyzed tissue spots: {total_spots}
- Spatial resolution: 10x Genomics Visium (55μm spots)

CELL TYPE COMPOSITION:
- Luminal epithelial cells (HR+): {luminal_pct:.1f}% of tissue
  * Primary subtype: LummHR-SCGB ({luminal_scgb} spots)
  * Secondary subtype: LummHR-major ({luminal_major} spots)
  * Key markers: {', '.join(top_luminal_markers)}

- Plasma cells (IgG+): {plasma_pct:.1f}% of tissue
  * Count: {plasma} spots
  * Key markers: {', '.join(top_plasma_markers)}

SPATIAL ORGANIZATION:
- Tumor-immune interface: {interface_pct:.1f}% of tissue shows direct contact between tumor and immune cells
- Spatial clusters identified: {spatial_data['dataset_info']['n_clusters']} distinct tissue regions
- Spatially variable genes (Moran's I > 0.1): {spatial_data['spatial_statistics']['morans_i']['n_significant_genes']} genes
  * Top spatially clustered genes: {', '.join(top_morans)}

MOLECULAR FEATURES:
- Quality metrics: {spatial_data['qc_metrics']['mean_genes_per_spot']:.0f} genes/spot, {spatial_data['qc_metrics']['mean_mt_percent']:.1f}% mitochondrial

Generate a clinical report including:
1. Diagnosis and tumor classification
2. Immune microenvironment assessment
3. Spatial organization patterns
4. Clinical implications for treatment
5. Brief statement on analytical limitations

Report should be professional, concise, and suitable for inclusion in patient medical records.

CLINICAL PATHOLOGY REPORT:"""
    
    return prompt

prompt = create_clinical_prompt(spatial_features, celltype_summary)
print("Generated Prompt:")
print("=" * 80)
print(prompt)
print("=" * 80)

## 4. Generate Clinical Report

In [ ]:
def generate_report(tokenizer, model, prompt, max_length=400):
    """Generate clinical report using MedGemma."""
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    print("Generating clinical report...")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    report = full_output.split("CLINICAL PATHOLOGY REPORT:")[-1].strip()
    
    return report

clinical_report = generate_report(tokenizer, model, prompt)

print("\n" + "=" * 80)
print("GENERATED CLINICAL PATHOLOGY REPORT")
print("=" * 80)
print(clinical_report)
print("=" * 80)
print(f"\nWord count: {len(clinical_report.split())}")

## 5. Save Report (TXT and JSON)

In [ ]:
def save_report(report, spatial_data, celltype_data, output_dir="../outputs"):
    """Save clinical report as TXT and structured JSON."""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    txt_path = f"{output_dir}/clinical_report_{timestamp}.txt"
    with open(txt_path, 'w') as f:
        f.write("SPATIAL TRANSCRIPTOMICS CLINICAL PATHOLOGY REPORT\n")
        f.write("=" * 80 + "\n\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Analysis Method: 10x Genomics Visium + MedGemma-4b-it\n")
        f.write(f"Specimen Type: Breast Cancer Biopsy\n\n")
        f.write("REPORT:\n")
        f.write("-" * 80 + "\n")
        f.write(report)
        f.write("\n" + "=" * 80 + "\n")
    
    report_data = {
        "metadata": {
            "timestamp": datetime.now().isoformat(),
            "model": "google/medgemma-4b-it",
            "quantization": "4-bit (NF4)",
            "word_count": len(report.split())
        },
        "clinical_report": report,
        "input_data_summary": {
            "total_spots": spatial_data['dataset_info']['total_spots'],
            "n_clusters": spatial_data['dataset_info']['n_clusters'],
            "n_spatially_variable_genes": spatial_data['spatial_statistics']['morans_i']['n_significant_genes'],
            "cell_type_composition": celltype_data['cell_type_composition'],
            "tumor_immune_interface_pct": celltype_data['tumor_immune_interface']['interface_pct']
        }
    }
    
    json_path = f"{output_dir}/clinical_report_{timestamp}.json"
    with open(json_path, 'w') as f:
        json.dump(report_data, f, indent=2)
    
    print(f"\nReports saved:")
    print(f"  TXT: {txt_path}")
    print(f"  JSON: {json_path}")
    
    return txt_path, json_path

txt_file, json_file = save_report(clinical_report, spatial_features, celltype_summary)

## 6. Memory Usage Check

In [ ]:
import psutil

process = psutil.Process()
memory_gb = process.memory_info().rss / 1024**3

print(f"\nMemory Usage:")
print(f"  Current process: {memory_gb:.2f} GB")
print(f"  Status: {'✓ Within 64GB limit' if memory_gb < 32 else '⚠ High memory usage'}")

if torch.backends.mps.is_available():
    print(f"  Device: Apple M1 MPS (Metal Performance Shaders)")

## Summary

Successfully generated clinical pathology report from spatial transcriptomics data:

1. Loaded spatial features and cell type annotations
2. Configured MedGemma-4b-it with 4-bit quantization
3. Generated clinical-quality report (~200 words)
4. Saved outputs as TXT and JSON
5. Verified memory usage <32GB

**Next Steps**: Deploy as Streamlit app with file upload interface